# Google Calendar Event Fetcher

#### The Notebook demonstrates how to programmatically retrieve and process Google Calendar events for a given user and date range.

##### Key Steps:

Authentication: Load user credentials from a token file.

API Call: Fetch events between specified start/end dates using the Google Calendar API.

Data Processing: Extract event details (start/end times, attendees) and structure them into a clean format.

Output: Return a list of events with attendee counts and time slots.
```

### Importing Required Libraries

In [57]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

### Defining the retrive_calendar_events Function that includes : 
#### 1. Authenticating with Google Calendar
#### 2. Fetching Events
#### 3. Processing Events & Structuring

In [76]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events :
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        EventId = event['id']
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"],
             "Id": EventId})
    return events_list

### Calling ```retrive_calendar_events``` with Auth Token, Start Date & End Date 
#### Date Format : YYYY-MM-DD (T) HH:MM:SS +TIMEZONE (+5:30 Indicates IST Time Zone Asia/Kolkata ) 

In [77]:
event = retrive_calendar_events("userthree.amd@gmail.com", '2025-07-24T00:00:00+05:30', '2025-07-24T23:59:59+05:30')

### Output that describing sample event : 

In [78]:
event

[{'StartTime': '2025-07-23T16:00:00+05:30',
  'EndTime': '2025-07-24T07:30:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours',
  'Id': '72l3ofl46ubh45k4gotqtbo9ci_20250723T103000Z'},
 {'StartTime': '2025-07-24T10:30:00+05:30',
  'EndTime': '2025-07-24T11:00:00+05:30',
  'NumAttendees': 3,
  'Attendees': ['usertwo.amd@gmail.com',
   'userthree.amd@gmail.com',
   'userone.amd@gmail.com'],
  'Summary': 'Agentic AI Project Status Update',
  'Id': 'v852me2e8c247edjitdfjt8m00'},
 {'StartTime': '2025-07-24T16:00:00+05:30',
  'EndTime': '2025-07-25T07:30:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours',
  'Id': '72l3ofl46ubh45k4gotqtbo9ci_20250724T103000Z'}]

[[{'StartTime': '2025-07-17T18:00:00+05:30', 'EndTime': '2025-07-18T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-18T18:00:00+05:30', 'EndTime': '2025-07-19T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}], [{'StartTime': '2025-07-17T18:00:00+05:30', 'EndTime': '2025-07-18T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-18T09:00:00+05:30', 'EndTime': '2025-07-18T17:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Workshop Advance AI'}, {'StartTime': '2025-07-18T18:00:00+05:30', 'EndTime': '2025-07-19T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]]
Free slots where both people are available:
1. 17:00 - 18:00 (2 people available)
